# 12. Creating a DataStore from numpy arrays
The goal of this notebook is to demonstrate how to create a `DataStore` from scratch. This can be useful if your device is not supported or if you would like to integrate the `dtscalibration` library in your current routine.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import xarray as xr

from dtscalibration import DataStore, read_silixa_files

For a `DataStore` object, a few things are needed:

- timestamps

- Stokes signal

- anti-Stokes signal

- x (length along fiber)

Let's grab the data from an existing silixa dataset:

In [ ]:
filepath = os.path.join('..', '..', 'tests', 'data', 'single_ended')

ds_silixa = read_silixa_files(directory=filepath,
                              silent=True)

We will get all the numpy arrays from this `DataStore` to create a new one from 'scratch'.

Let's start with the most basic data:

In [ ]:
x = ds_silixa.x.values
time = ds_silixa.time.values
ST = ds_silixa.st.values
AST = ds_silixa.ast.values

Now this data has to be inserted into an xarray `Dataset`

In [ ]:
ds = xr.Dataset()
ds['x'] = ('x', x)
ds['time'] = ('time', time)
ds['st'] = (['x', 'time'], ST)
ds['ast'] = (['x', 'time'], AST)

In [ ]:
ds = DataStore(ds)
print(ds)

For calibration, a few more paramaters are needed:

- acquisition time (for calculating residuals for WLS calibration)

- reference temperatures

- a double ended flag

We'll put these into the custom `DataStore`:

In [ ]:
ds['acquisitiontimeFW'] = ds_silixa['acquisitiontimeFW'].values
ds['temp1'] = ds_silixa['probe1Temperature']
ds['temp2'] = ds_silixa['probe2Temperature']

ds.attrs['isDoubleEnded'] = '0'

Now we can calibrate the data as usual (ordinary least squares in this example).

In [ ]:
ds = ds.sel(x=slice(-30, 101))
sections = {
            'temp1':    [slice(20, 25.5)],  # warm bath
            'temp2':    [slice(5.5, 15.5)],  # cold bath
            }
ds.sections = sections

ds.calibration_single_ended(method='ols')

ds.isel(time=0).tmpf.plot()